## Customer Lifetime Value Prediction - Credit Card Users

<img src="clv.png" alt="Drawing" style="width: 500px;"/>

### What is Customer Lifetime value?

In marketing, customer lifetime value (CLV or often CLTV), lifetime customer value (LCV), or life-time value (LTV) is a prediction of the net profit attributed to the entire future relationship with a customer. Customer lifetime value is a powerful piece of business intelligence that informs an efficient strategy for business growth.

### Why is Customer Lifetime Value Important?

1) Customer Lifetime Value (CLV) helps you allocate your customer procurement budget based on what the new client will actually bring to your firm.

2) CLV data helps build more accurate and detailed customer personas. 

3) To  Learn the effect of certain high-level decisions on the value of customer assets. The CLV data can be used to encourage a company culture emphasizing long-term customer satisfaction, rather than solely focusing on short-term sales.

4) You can calculate the profitability of a single customer. 

### How machine learning helps?

Because of the importance of CLV, it is essential to know the future value of customers by looking at their historical transactions and hence look at the revenue of the organization as a whole. Machine Learning helps in bridging this gap by learning from historical data and predicting the following:

1) The persona and the approximate Customer Lifetime Value of a new customer.

2) Predicting the next transaction amounts and periodicity for existing customers, hence giving an indication of their adjusted CLV.

### Case 1: CLV Machine Learning Models for New Customers

As part of the first use case, this notebook will walk through how historical customer demographic and transaction data is used to build a model to predict the persona and approximate CLV for a new customer.

#### Understanding the data

Let us have a look at some sample data which defines a customer and the transaction history of a customer.

In [3]:
import pandas as pd

# Reading sample data
sample_data = pd.read_csv('..\\99_sample_data\\custclv.csv')

# Showing top 10 records
sample_data.head(10)

,customer_id,rfm_score,Gender,Age,HouseType,ContactAvaliability,HomeCountry,CreditScore,CLV,MonthlyValue,ActiveMonths
0,10235005,322,Male,18,Owned,Medium,Singapore,NaN,7.3780,3.689000,2
1,10238628,535,Male,37,Rented,Low,Singapore,NaN,399.1002,199.550100,2
2,10256519,545,Male,50,Owned,High,Singapore,NaN,534.6880,534.688000,1
3,10261907,213,Male,57,Owned,Low,Singapore,NaN,17.9180,17.918000,1
4,10269642,554,Male,28,Owned,High,Singapore,NaN,570.9394,142.734850,4
5,10282193,111,Male,50,Rented,High,Singapore,NaN,0.0000,0.000000,1
6,10285129,115,Male,38,Rented,High,Singapore,NaN,65.6084,65.608400,1
7,10294523,543,Male,46,Owned,High,Singapore,NaN,97.4826,32.494200,3
8,10307950,554,Female,38,Owned,Medium,Singapore,NaN,379.1300,126.376667,3
9,10308730,211,Female,8,Rented,High,Singapore,NaN,0.0000,0.000000,1


You can see that the data consists of the demographic information of the customer like Gender, Age, Country and so on and also features derived from the transaction data such as RFM, CLV, Avg Monthly Value of the Customer, Active Months among others. This data is critical for machine learning. 

#### Building the machine learning model

In order to build the machine learning model we make use of some of the libraries available within the H2O.ai framework. These models are extremely efficient and optimize the machine learning models. For this use case we will make use of the H2O KMeans Estimator to segment the customer into different buckets.

In [17]:
from h2o.estimators.kmeans import H2OKMeansEstimator
import h2o

h2o.init(ip="127.0.0.1", https=True, insecure = True) # initializing h2o
predictors = ['rfm_score','Gender','Age','HouseType','ContactAvailability','HomeCountry',
              'CreditScore','CLV','MonthlyValue','ActiveMonths']
    
# Data Cleaning
sample_data['Gender'] = pd.Categorical(sample_data.Gender).codes
sample_data['ContactAvaliability'] = pd.Categorical(sample_data.ContactAvaliability).codes
sample_data['HouseType'] = pd.Categorical(sample_data.HouseType).codes
sample_data['HomeCountry'] = pd.Categorical(sample_data.HomeCountry).codes
sample_data_h20 = h2o.H2OFrame(sample_data)
sample_data_h20_test = sample_data_h20

Checking whether there is an H2O instance running at https://127.0.0.1:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from c:\python35\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\aditya.am\AppData\Local\Temp\tmp26qzuiov
  JVM stdout: C:\Users\aditya.am\AppData\Local\Temp\tmp26qzuiov\h2o_aditya_am_started_from_python.out
  JVM stderr: C:\Users\aditya.am\AppData\Local\Temp\tmp26qzuiov\h2o_aditya_am_started_from_python.err
  Server is running at http://127.0.0.1:54325
Connecting to H2O server at http://127.0.0.1:54325... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,Asia/Singapore
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.6
H2O cluster version age:,5 days
H2O cluster name:,H2O_from_python_aditya_am_j4r2wu
H2O cluster total nodes:,1
H2O cluster free memory:,1.712 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


c:\python35\lib\site-packages\h2o\utils\shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [18]:
# Training the Machine Learning Model
estimator = H2OKMeansEstimator(k=4, init="Random", seed=2, standardize=True, categorical_encoding='auto')
estimator.train(x=predictors,training_frame=sample_data_h20)
trained = estimator.predict(sample_data_h20)
sample_data_h20['cluster'] = trained["predict"].asfactor()
sample_data_h20 = sample_data_h20.as_data_frame()
sample_data_h20.head(10) # Visualizing the output

kmeans Model Build progress: |████████████████████████████████████████████| 100%
kmeans prediction progress: |█████████████████████████████████████████████| 100%


,customer_id,rfm_score,Gender,Age,HouseType,ContactAvaliability,HomeCountry,CreditScore,CLV,MonthlyValue,ActiveMonths,ContactAvailability,cluster
0,10235005,322,1,18,0,2,0,NaN,7.3780,3.689000,2,2,3
1,10238628,535,1,37,1,1,0,NaN,399.1002,199.550100,2,1,2
2,10256519,545,1,50,0,0,0,NaN,534.6880,534.688000,1,0,1
3,10261907,213,1,57,0,1,0,NaN,17.9180,17.918000,1,1,3
4,10269642,554,1,28,0,0,0,NaN,570.9394,142.734850,4,0,1
5,10282193,111,1,50,1,0,0,NaN,0.0000,0.000000,1,0,0
6,10285129,115,1,38,1,0,0,NaN,65.6084,65.608400,1,0,0
7,10294523,543,1,46,0,0,0,NaN,97.4826,32.494200,3,0,1
8,10307950,554,0,38,0,2,0,NaN,379.1300,126.376667,3,2,1
9,10308730,211,0,8,1,0,0,NaN,0.0000,0.000000,1,0,0


#### Predicting CLV for a new customer

Using the above built model let us try to predict the CLV for a new customer.

In [22]:
# For a new customer
sample_customer = sample_data_h20_test[2,:]
predicted = estimator.predict(sample_customer)
print("Predicted Cluster : ",predicted["predict"].asfactor())

# Calculate average CLV of that cluster
req_value = int(predicted["predict"].asfactor())
req_data = sample_data_h20[sample_data_h20['cluster']==req_value]
avg_clv = req_data['CLV'].mean()
avg_RFM = req_data['rfm_score'].mean()
print('CLV for new customer = ',str(avg_clv))
print('RFM for new customer = ',str(int(avg_RFM)))

kmeans prediction progress: |█████████████████████████████████████████████| 100%
Predicted Cluster :  

predict
1



CLV for new customer =  723.23124
RFM for new customer =  485


Based on the above predited cluster, we can find out and build the persona of the customer based on all the other customers existing in that cluster. The below function is used to build the persona and the output can be obtained as shown below. This is important to know since it gives a 360 degree view of the customer which allows the organization to better understand the customer.

In [ ]:
# Function to estimate the persona of a new customer
customer_persona = personaestimator(sample_customer)

customer_persona

Customer is a high spend customer
Customer spends very frequently
Customer likes to spend with Restaurants and Airlines
Customer is similar to the age range of 30-37

The above information is very critical for an organization to get a full view and understand the customer better. In order to understand the complete process of this use case please visit our blog at : <a href="https://www.justanalytics.com/blog/customer-lifetime-value-how-to-identify-your-most-important-customers">Customer Lifetime Value - How to identify your most important customers<a>

### Case 2: Predicting CLV for Existing Customer

It is important for an organization to know the CLV of every customer to as future looking an extent as possible in order to acquire information and decide on how much to spend on that customer to keep him/her with the organization.

Deep learning comes in handy in order to predict further transactions of a customer in order to know theadjusted CLV at any point in time. It does this by learning about the customer from the transactions and demographic data. Two things are predicted which contribute towards calculation of the predicted CLV.

1) Prediction of when the future customer transactions will take place

2) Prediction of the transaction amount of the future transactions

In [23]:
sample_case2_data = pd.read_csv('..\\99_sample_data\\customerpreddata.csv')
sample_case2_data.head(10)

,AccountNo,Gender,Age,HouseType,ContactAvailability,HomeCountry,CreditScore,CardExpiryDate,TransactionCountry,LargePurchase,Product,TransactionCurrencyCode,date_diff,Card_Age,Week_Segment,TransactionAmount,lag_Datediff,lag_Txamt
0,1308303,Female,56,Owned,Medium,Singapore,1,1908,Singapore,0,1,SGD,0,300,WEEKDAY,12.400,0.0,3.100
1,1308303,Female,56,Owned,Medium,Singapore,1,1908,Singapore,0,1,SGD,0,305,WEEKEND,3.100,5.0,1.240
2,1308303,Female,56,Owned,Medium,Singapore,1,1908,Singapore,0,1,SGD,5,305,WEEKEND,1.240,2.0,6.200
3,1563687,Male,49,Owned,Low,Singapore,1,2003,Singapore,0,1,SGD,0,130,WEEKEND,10.044,256.0,8.680
4,1563687,Male,49,Owned,Low,Singapore,1,2003,Singapore,0,1,SGD,256,386,WEEKDAY,8.680,NaN,NaN
5,7131505,Male,57,Owned,Medium,Singapore,1,2006,Singapore,0,1,SGD,0,59,WEEKDAY,25.420,0.0,25.420
6,7131505,Male,57,Owned,Medium,Singapore,1,2006,Singapore,0,1,SGD,0,59,WEEKDAY,25.420,0.0,25.420
7,7131505,Male,57,Owned,Medium,Singapore,1,2006,Singapore,0,1,SGD,0,59,WEEKDAY,25.420,0.0,17.050
8,7131505,Male,57,Owned,Medium,Singapore,1,2006,Singapore,0,1,SGD,0,59,WEEKDAY,17.050,109.0,4.743
9,7150411,Male,54,Owned,Medium,Singapore,1,2007,Singapore,0,1,SGD,0,93,WEEKEND,9.579,0.0,9.579


The data being used for this case consists of the demographic data of the customer, credit card product details of the customer, transaction details (amount, date, time) and other derived features. These details are pivotal for this machine learning activity.

### Train a deep learning model

For the training exercise we choose a deep neural network model to learn from the data and be used to predict what we spoke of before. For this notebook we will be skipping how the model is created, rather focus on using the built model and focus on scoring. 

### Scoring using the trained model

We use the above trained model to predict for a particular customer the amount and the days for the next transaction. In an ideal case this would be done for the next 'n' transactions.

#### Predicting the next transaction amount and days

In [46]:
from keras.models import model_from_json

############################# load json and create model
json_file = open('..\\05_models\\clv_amt_dl_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("..\\05_models\\clv_amt_dl_model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam')
#########################


############################# load json and create model
json_file = open('..\\05_models\\clv_days_dl_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_days = model_from_json(loaded_model_json)
# load weights into new model
loaded_model_days.load_weights("..\\05_models\\clv_days_dl_model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model_days.compile(loss='binary_crossentropy', optimizer='adam')
#########################

Loaded model from disk
Loaded model from disk


In [44]:
all_data = sample_case2_data
all_data = all_data[all_data['AccountNo']==7131505]

# Convert all data to categorical
all_data = all_data.drop('lag_Datediff',1)
all_data['Gender'] = pd.Categorical(all_data.Gender).codes
all_data['ContactAvailability'] = pd.Categorical(all_data.ContactAvailability).codes
all_data['HouseType'] = pd.Categorical(all_data.HouseType).codes
all_data['HomeCountry'] = pd.Categorical(all_data.HomeCountry).codes
all_data['TransactionCountry'] = pd.Categorical(all_data.TransactionCountry).codes
all_data['TransactionCurrencyCode'] = pd.Categorical(all_data.TransactionCurrencyCode).codes
all_data['Week_Segment'] = pd.Categorical(all_data.Week_Segment).codes  
all_data['Product'] = pd.Categorical(all_data.Product).codes  
X = all_data.iloc[0:len(all_data)-1,1:15] # dont consider account no
Y = all_data.iloc[0:len(all_data)-1,16] # lag transaction amount column

loaded_model.fit(X, Y, epochs=100, verbose=0)
Xnew = all_data.tail(1) # dont consider account no
Xnew = Xnew.iloc[:,1:15]
ynew = loaded_model.predict(Xnew)
print("Predicted Value (Amount)  = {}".format(str(abs(ynew[0]))))

Predicted Value (Amount)  = [10.368908]


In [50]:
all_data = sample_case2_data
all_data = all_data[all_data['AccountNo']==7131505]

# Convert all data to categorical
all_data = all_data.drop('lag_Txamt',1)
all_data['Gender'] = pd.Categorical(all_data.Gender).codes
all_data['ContactAvailability'] = pd.Categorical(all_data.ContactAvailability).codes
all_data['HouseType'] = pd.Categorical(all_data.HouseType).codes
all_data['HomeCountry'] = pd.Categorical(all_data.HomeCountry).codes
all_data['TransactionCountry'] = pd.Categorical(all_data.TransactionCountry).codes
all_data['TransactionCurrencyCode'] = pd.Categorical(all_data.TransactionCurrencyCode).codes
all_data['Week_Segment'] = pd.Categorical(all_data.Week_Segment).codes  
all_data['Product'] = pd.Categorical(all_data.Product).codes  
X = all_data.iloc[0:len(all_data)-1,1:15] # dont consider account no
Y = all_data.iloc[0:len(all_data)-1,16] # lag transaction amount column

loaded_model_days.fit(X, Y, epochs=100, verbose=0)
Xnew = all_data.tail(1) # dont consider account no
Xnew = Xnew.iloc[:,1:15]
ynew = loaded_model_days.predict(Xnew)
print("Predicted Value (Days) = {}".format(str(abs(ynew[0]))))

Predicted Value (Days) = [92.0834]


So, from the above scoring models, it can be seen that for a sample customer, with customer ID 7131505

Predicted Value (Amount)  = [10.368908] -> Next transaction amount

Predicted Value (Days) = [92.0834] -> When the next transaction will happen

Using these values we can recalculate the CLV from time to time and keep the organization data updated.

### Conclusion

In conclusion, through this notebooks, we understood the importance of know your customer lifetime value and how deep learning models can help in predicting the CLV of a customer and also how a new customer can be tied to the business right from the start.

In order to know, how we can improve the CLV of a customer and hence his retention is the company, head over to our notebook <a href="recommendation_ml.ipynb">Recommendation IPython Notebook.</a>